## Imagine que queremos adicionar os seguintes atributos:
* **Winner**: receberá 1 ou 0, em casos do time ter vencido ou perdido a partida, respectivamente;
* **Dominant Team**: receberá 1 ou 0, em casos do time ter sido dominante ou não, respectivamente;
* **Position Ranking Fifa**: receberá a posição do [ranking da fifa](https://www.fifa.com/fifa-world-ranking/ranking-table/men/index.html) de melhores seleções do mundo.
    * O time dominente é aquele que tem uma posso de bola, chutes no alvo e número de passes certos acima da média
* **Already World Champean**: receberá 1 ou 0, em casos do time ter vencido uma edição anterior da copa do mundo ou não, respectivamente;

In [165]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

plt.rc('figure', figsize = (20, 8))
df = pd.read_csv('data/FIFA_2018_Statistics_sem_NaN.csv')
df.head()

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,0,0,1,12.0,Group Stage,0,0,0.0,0.0
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,0,0,-1.0,Group Stage,0,0,0.0,0.0
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,2,0,0,0,-1.0,Group Stage,0,0,0.0,0.0
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,0,0,1,89.0,Group Stage,0,0,0.0,0.0
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,1,0,0,0,-1.0,Group Stage,0,0,1.0,90.0


# Adicionando novos Atributos

## Atributo winner

In [166]:
df['Winner'] = -1

for i in range(0,df.shape[0], 2):
    if (df.loc[i, 'Goal Scored'] > df.loc[i+1, 'Goal Scored']):
        df.loc[i, 'Winner'] = 1
        df.loc[i+1, 'Winner'] = 0
    elif (df.loc[i, 'Goal Scored'] < df.loc[i+1, 'Goal Scored']):
        df.loc[i, 'Winner'] = 0
        df.loc[i+1, 'Winner'] = 1
    else:
        df.loc[i, 'Winner'] = 2
        df.loc[i+1, 'Winner'] = 2

df[['Team', 'Opponent', 'Winner']].head()

,Team,Opponent,Winner
0,Russia,Saudi Arabia,1
1,Saudi Arabia,Russia,0
2,Egypt,Uruguay,0
3,Uruguay,Egypt,1
4,Morocco,Iran,0


### Atributo Dominant Team

In [167]:
df['Dominant Team'] = (df['Ball Possession %'] > df['Ball Possession %'].mean()) & (df['On-Target']>df['On-Target'].mean()) & (df['Pass Accuracy %'] > df['Pass Accuracy %'].mean())
df['Dominant Team'] = df['Dominant Team'].map({True: 1, False: 0})
df.head()

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time,Winner,Dominant Team
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,1,12.0,Group Stage,0,0,0.0,0.0,1,0
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,-1.0,Group Stage,0,0,0.0,0.0,0,0
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,0,0,-1.0,Group Stage,0,0,0.0,0.0,0,0
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,1,89.0,Group Stage,0,0,0.0,0.0,1,1
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,0,0,-1.0,Group Stage,0,0,1.0,90.0,0,0


### Lendo ranking da Fifa

In [168]:
ls_fifa = pd.read_html('web_scrapping/The FIFA_Coca-Cola World Ranking - Ranking Table - FIFA.com.html')

In [206]:
df_rank_fifa = ls_fifa[0]
df_rank_fifa = df_rank_fifa[['2018 (100 %)', '2017 (50 %)']]
#filtrando e renomeando as colunas
df_rank_fifa.columns=['ranking', 'Team']
df_rank_fifa.head(40)

,ranking,Team
0,1,Germany
1,2,Brazil
2,3,Belgium
3,4,Portugal
4,5,Argentina
5,6,Switzerland
6,7,France
7,8,Poland
8,9,Chile
9,10,Spain


* O nome Iran está 'IR Iran' no ranking da Fifa. Vamos substituir usando a função **replace**.

In [207]:
df_rank_fifa[df_rank_fifa['Team'] == 'IR Iran']

,ranking,Team
36,37,IR Iran


In [208]:
#df_rank_fifa['Team'] = df_rank_fifa['Team'].replace(['IR Iran'], ['Iran'])
#df_rank_fifa['Team'] = df_rank_fifa['Team'].replace(['IR Iran'], ['Iran'])
df_rank_fifa.loc[36,'Team'] = 'Iran'
df_rank_fifa.head(40)

/home/dwan/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,ranking,Team
0,1,Germany
1,2,Brazil
2,3,Belgium
3,4,Portugal
4,5,Argentina
5,6,Switzerland
6,7,France
7,8,Poland
8,9,Chile
9,10,Spain


* De todos os paízes que aparecem no ranking da Fifa, queremos apenas os que participaram da copa 2018
* Faremos um filtro utilizando a função **isin**

In [209]:
s = list(df.Team.unique())
mask = df_rank_fifa['Team'].isin(values=s)
df_rank_fifa = df_rank_fifa[mask]
df_rank_fifa

,ranking,Team
0,1,Germany
1,2,Brazil
2,3,Belgium
3,4,Portugal
4,5,Argentina
5,6,Switzerland
6,7,France
7,8,Poland
9,10,Spain
10,11,Peru


* Agora vamos adicionar o atributo 'Position Ranking Fifa'
    * Para tanto, iremos acessar seleção por seleção no nosso dataframe **df**;
    * Depois acessaremos a posicao do rank desta seleção
        * Note que ao acessarmos a função loc para encontrar o ranking da seleção, ela retorna uma series, por isso usamos o iat[0], conforme recomendado no blog de [Mark Needham](https://markhneedham.com/blog/2013/10/30/pandas-adding-a-column-to-a-dataframe-based-on-another-dataframe/)

In [210]:
df['Position Ranking Fifa'] = -1
for i in range(df.shape[0]):
    pais = df.loc[i, 'Team']
    rank = df_rank_fifa.loc[df_rank_fifa['Team']==pais, 'ranking'].iat[0]
    df.loc[i, 'Position Ranking Fifa'] = rank
    #df.loc[i, 'Position Ranking Fifa'] = df_rank_fifa.loc[df_rank_fifa['Team']==pais, 'ranking'].iat[0]
df

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time,Winner,Dominant Team,Position Ranking Fifa
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,1,12.0,Group Stage,0,0,0.0,0.0,1,0,70
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,-1.0,Group Stage,0,0,0.0,0.0,0,0,67
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,0,-1.0,Group Stage,0,0,0.0,0.0,0,0,45
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,1,89.0,Group Stage,0,0,0.0,0.0,1,1,14
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,0,-1.0,Group Stage,0,0,1.0,90.0,0,0,41
5,15-06-2018,Iran,Morocco,1,36,8,2,5,1,2,...,1,90.0,Group Stage,0,0,0.0,0.0,1,0,37
6,15-06-2018,Portugal,Spain,3,39,8,3,2,3,4,...,0,4.0,Group Stage,0,0,0.0,0.0,2,0,4
7,15-06-2018,Spain,Portugal,3,61,12,5,5,2,5,...,1,24.0,Group Stage,0,0,0.0,0.0,2,1,10
8,16-06-2018,France,Australia,2,51,12,5,4,3,5,...,1,58.0,Group Stage,0,0,0.0,0.0,1,1,7
9,16-06-2018,Australia,France,1,49,4,1,2,1,1,...,0,62.0,Group Stage,0,0,1.0,81.0,0,0,36


## Atributo Already World Champeon

* Para tanto vamos usar a função apply que aplica uma função passada como parâmetro a cada registro do DataFrame.
* Iremos usar uma função lambda para este fim;

In [213]:
world_champeons = ['Germany', 'Brazil', 'Argentina', 'France', 
                   'Spain', 'England', 'Uruguay']
df['Already World Champeon'] = df['Team'].apply(
    func = lambda selecao: 1 if selecao in world_champeons else 0)

In [229]:
df.head()

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time,Winner,Dominant Team,Position Ranking Fifa,Already World Champeon
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,12.0,Group Stage,0,0,0.0,0.0,1,0,70,0
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,-1.0,Group Stage,0,0,0.0,0.0,0,0,67,0
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,-1.0,Group Stage,0,0,0.0,0.0,0,0,45,0
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,89.0,Group Stage,0,0,0.0,0.0,1,1,14,1
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,-1.0,Group Stage,0,0,1.0,90.0,0,0,41,0


## Exportando o dataset

In [230]:
df.to_csv('data/FIFA_2018_Statistics_sem_NaN_novos_Atributos.csv', index=False)

# Removendo atributos

* Iremos utilizar a função **drop**. 
    * Na função drop, axis=1 significa que queremos remover colunas
* A mesma tarefa poderia ser realizada utilizando o comando **del** ou a função pop().

In [216]:
colunas = ['Attempts', 'On-Target', 'Off-Target', 'Blocked', 'Corners', 'Offsides']
df_new = df.drop(labels = colunas, axis=1)
df_new.head()

,Date,Team,Opponent,Goal Scored,Ball Possession %,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),...,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time,Winner,Dominant Team,Position Ranking Fifa,Already World Champeon
0,14-06-2018,Russia,Saudi Arabia,5,40,11,0,78,306,118,...,12.0,Group Stage,0,0,0.0,0.0,1,0,70,0
1,14-06-2018,Saudi Arabia,Russia,0,60,25,2,86,511,105,...,-1.0,Group Stage,0,0,0.0,0.0,0,0,67,0
2,15-06-2018,Egypt,Uruguay,0,43,7,3,78,395,112,...,-1.0,Group Stage,0,0,0.0,0.0,0,0,45,0
3,15-06-2018,Uruguay,Egypt,1,57,13,3,86,589,111,...,89.0,Group Stage,0,0,0.0,0.0,1,1,14,1
4,15-06-2018,Morocco,Iran,0,64,14,2,86,433,101,...,-1.0,Group Stage,0,0,1.0,90.0,0,0,41,0


## Dica

Normalmente quando temos um dataframe e vamos utilizando para praticar aprendizagem de máquina supervisionada usando o scikit-learn, a linha seguinte é bastante comum para separar as variáveis independentes da variável dependente (classe)

In [227]:
X, y = df_new.drop(['Winner'], axis=1).values, df_new['Winner'].values